## Creating database

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('db_inventory.db')
c = conn.cursor()

# Create tables in SQLite
c.execute('''
CREATE TABLE Suppliers (
    supplier_id INTEGER PRIMARY KEY,
    supplier_name TEXT NOT NULL
)
''')

c.execute('''
CREATE TABLE Products (
    product_code INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    safety_stock INTEGER,
    unit_height INTEGER,
    unit_width INTEGER,
    unit_depth INTEGER,
    unit_volume DECIMAL(5, 2)
)
''')

c.execute('''
CREATE TABLE Warehouses (
    warehouse_id INTEGER PRIMARY KEY,
    warehouse_name TEXT NOT NULL,
    capacity INTEGER
)
''')

c.execute('''
CREATE TABLE WarehouseStock (
    stock_id INTEGER PRIMARY KEY,
    stock_date DATE,
    product_code INTEGER,
    warehouse_id INTEGER,
    quantity_on_hand INTEGER,
    quantity_reserved INTEGER,
    quantity_available INTEGER,
    FOREIGN KEY (product_code) REFERENCES Products(product_code),
    FOREIGN KEY (warehouse_id) REFERENCES Warehouses(warehouse_id)
)
''')

c.execute('''
CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT NOT NULL
)
''')

c.execute('''
CREATE TABLE Orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    expected_shipping_date DATE,
    shipping_date DATE,
    shipping_status TEXT,
    sales_channel TEXT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
)
''')

c.execute('''
CREATE TABLE OrderDetails (
    order_product_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_code INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_code) REFERENCES Products(product_code)
)
''')

c.execute('''
CREATE TABLE Deliveries (
    delivery_id INTEGER PRIMARY KEY,
    supplier_id INTEGER,
    order_date DATE,
    expected_delivery_date DATE,
    delivery_date DATE,
    delivery_status TEXT,
    FOREIGN KEY (supplier_id) REFERENCES Suppliers(supplier_id)
)
''')

c.execute('''
CREATE TABLE DeliveryDetails (
    delivery_detail_id INTEGER PRIMARY KEY,
    delivery_id INTEGER,
    product_code INTEGER,
    quantity INTEGER,
    FOREIGN KEY (delivery_id) REFERENCES Deliveries(delivery_id),
    FOREIGN KEY (product_code) REFERENCES Products(product_code)
)
''')

# Save changes and close the connection
conn.commit()
conn.close()

print("The database and tables have been successfully created.")


## Adding first records to database

In [ ]:
import sqlite3
import pandas as pd
import os

def display_added_records(num_added_records, table_name):
    print(f"{num_added_records} new records were added to the '{table_name}' table.")

def add_new_records(conn, excel_file, table_name):
    # Retrieve existing records from the specified table as a DataFrame
    existing_data = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Load data from the Excel file located in the "data" folder
    new_data = pd.read_excel(excel_file)

    # Select only those records from the Excel file that do not already exist in the database
    new_records = new_data[~new_data.iloc[:, 0].isin(existing_data.iloc[:, 0])]

    num_added_records = len(new_records)

    # Add the new records to the database
    new_records.to_sql(table_name, conn, if_exists='append', index=False)

    return num_added_records

# Connect to the SQLite database
conn = sqlite3.connect('db_inventory.db')

# List of Excel files and their corresponding SQL tables
excel_files = {
    'suppliers.xlsx': 'Suppliers',
    'products.xlsx': 'Products',
    'warehouses.xlsx': 'Warehouses',
    'warehouse_stock.xlsx': 'WarehouseStock',
    'customers.xlsx': 'Customers',
    'orders.xlsx': 'Orders',
    'order_details.xlsx': 'OrderDetails',
    'deliveries.xlsx': 'Deliveries',
    'delivery_details.xlsx': 'DeliveryDetails'
}

# Add new records from each Excel file to the corresponding SQL tables
for excel_file, table_name in excel_files.items():
    excel_file_path = os.path.join("data", excel_file)
    if os.path.exists(excel_file_path):
        num_added_records = add_new_records(conn, excel_file_path, table_name)
        display_added_records(num_added_records, table_name)
    else:
        print(f"The file '{excel_file}' does not exist. Skipping...")

# Close the database connection
conn.close()
